In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.mn2graph as m2g

# Repr et str

In [2]:
gum.config.reset() # back to default
mn=gum.fastMN("A-B-C;C-D;B-E-F;F-D-G;H-J;E-A;J")
mn

(gum::MarkovNet<double>@0x7fca4e996c00) MN{nodes: 9, edges: 12, domainSize: 512, dim: 38}

In [3]:
gum.config.reset() # back to default
gum.config['factorgraph','edge_length']='0.4'
mn

(gum::MarkovNet<double>@0x7fca4e996c00) MN{nodes: 9, edges: 12, domainSize: 512, dim: 38}

In [4]:
gum.config.reset() # back to default
gum.config['notebook','default_markovnetwork_view']='graph'
mn

(gum::MarkovNet<double>@0x7fca4e996c00) MN{nodes: 9, edges: 12, domainSize: 512, dim: 38}

In [5]:
print(mn)

MN{nodes: 9, edges: 12, domainSize: 512, dim: 38}


# accessors

In [6]:
print(f"nodes       : {mn.nodes()}")
print(f"node names  : {mn.names()}")
print(f"edges       : {mn.edges()}")
print(f"components  : {mn.connectedComponents()}")
print(f"factors     : {mn.factors()}")
print(f"factor(C,D) : {mn.factor({2,3})}")
print(f"factor(C,D) : {mn.factor({'C','D'})}")
print(f"factor(C,D) : {mn.factor({'D','C'})}")

nodes       : {0, 1, 2, 3, 4, 5, 6, 7, 8}
node names  : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
edges       : {(0, 1), (1, 2), (0, 4), (1, 5), (1, 4), (2, 3), (4, 5), (0, 2), (5, 6), (7, 8), (3, 6), (3, 5)}
components  : {0: {0, 1, 2, 3, 4, 5, 6}, 7: {8, 7}}
factors     : [{0, 1, 2}, {2, 3}, {1, 4, 5}, {3, 5, 6}, {8, 7}, {0, 4}, {8}]
factor(C,D) : <C:0|D:0> :: 0.922667 /<C:1|D:0> :: 0.257561 /<C:0|D:1> :: 0.824553 /<C:1|D:1> :: 0.255309
factor(C,D) : <C:0|D:0> :: 0.922667 /<C:1|D:0> :: 0.257561 /<C:0|D:1> :: 0.824553 /<C:1|D:1> :: 0.255309
factor(C,D) : <C:0|D:0> :: 0.922667 /<C:1|D:0> :: 0.257561 /<C:0|D:1> :: 0.824553 /<C:1|D:1> :: 0.255309


In [7]:
try:
    mn.factor({0,1})
except gum.GumException as e:
    print(e)
try:
    mn.factor({"A","B"})
except gum.GumException as e:
    print(e)

[pyAgrum] Object not found: No element with the key <{1,0}>
[pyAgrum] Object not found: No element with the key <{1,0}>


# Manipulating factors

In [8]:
mn.factor({'A','B','C'})

(gum::Potential<double>@0x7fca2e484de0) <A:0|B:0|C:0> :: 0.658307 /<A:1|B:0|C:0> :: 0.167962 /<A:0|B:1|C:0> :: 0.941589 /<A:1|B:1|C:0> :: 0.287343 /<A:0|B:0|C:1> :: 0.370682 /<A:1|B:0|C:1> :: 0.0450858 /<A:0|B:1|C:1> :: 0.756929 /<A:1|B:1|C:1> :: 0.711961

In [9]:
mn.factor({'A','B','C'})[{'B':0}]

array([[0.65830713, 0.16796225],
       [0.37068157, 0.04508579]])

In [10]:
mn.factor({'A','B','C'})[{'B':0}]=[[1,2],[3,4]]
mn.factor({'A','B','C'})

(gum::Potential<double>@0x7fca2e484de0) <A:0|B:0|C:0> :: 1 /<A:1|B:0|C:0> :: 2 /<A:0|B:1|C:0> :: 0.941589 /<A:1|B:1|C:0> :: 0.287343 /<A:0|B:0|C:1> :: 3 /<A:1|B:0|C:1> :: 4 /<A:0|B:1|C:1> :: 0.756929 /<A:1|B:1|C:1> :: 0.711961

# detailed graphical representation

In [19]:
gum.config.reset() # back to default
gum.config['factorgraph','edge_length']='0.5'

maxnei=max([len(mn.neighbours(n)) for n in mn.nodes()])
nodemap={n:len(mn.neighbours(mn.idFromName(n)))/maxnei for n in mn.names()}

facmax=max([len(f) for f in mn.factors()])
fgma=lambda factor: (1+len(factor)**2)/(1+facmax*facmax)

gnb.sideBySide(gnb.getGraph(m2g.MN2UGdot(mn),size="100"),
               gnb.getGraph(m2g.MN2UGdot(mn,nodeColor=nodemap),size="100"),
               gnb.getGraph(m2g.MN2FactorGraphdot(mn),size="100"),
               gnb.getGraph(m2g.MN2FactorGraphdot(mn,factorColor=fgma),size="100"),
               captions=['Markov network',
                         'MarkovNet with colored node w.r.t number of neighbours',
                         'Markovnet as factor graph',
                         'MN with colored factor w.r.t to the size of scope'])

Markov network,MarkovNet with colored node w.r.t number of neighbours,Markovnet as factor graph,MN with colored factor w.r.t to the size of scope


# from BayesNet to MarkovNet

In [31]:
bn=gum.fastBN("A->B<-C->D->E->F<-B<-G;A->H->I;C->J")
mn=gum.MarkovNet.fromBN(bn)
gnb.sideBySide(bn,mn,captions=['a Bayesian network','the corresponding Markov Network'])

a Bayesian network,the corresponding Markov Network


In [21]:
print(d.to_string())

graph G {
bgcolor=transparent;
layout=neato;
size=4;
"A" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(0) A", width=0];
"B" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(1) B", width=0];
"C" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(2) C", width=0];
"D" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(3) D", width=0];
"E" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(4) E", width=0];
"F" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(5) F", width=0];
"G" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(6) G", width=0];
"H" [fillcolor=coral, fontcolor=black, height=0, margin=0.04, shape=rectangle, style=filled, tooltip="(7) H